### 0️⃣ Initialize notebook variables

Configure the deployment parameters according to your preferences.

In [1]:
import os
import sys
import json
from utils import run_command, build_and_deploy_mcp_server, list_mcp_tools

deployment_name = "azure-architecture-toolkit"
resource_group_name = f"lab-{deployment_name}"
resource_group_location = "uksouth"  # Change to your preferred location

# Container images configuration
build_number = 0

# MCP Servers configuration - dynamic list of repositories
mcp_servers = [
    {
        "name": "avm-modules",
        "display_name": "AVM MCP Server",
        "repo_url": "https://github.com/Vinny614/avm-mcp-server.git",
        "image_name": "mcp-avm-modules"
    },
    {
        "name": "azure-pricing",
        "display_name": "Azure Pricing MCP Server",
        "repo_url": "https://github.com/Vinny614/mcp-azure-pricing.git",
        "image_name": "mcp-azure-pricing"
    },
    {
        "name": "markitdown",
        "display_name": "Markitdown MCP Server",
        "image_name": "mcp-markitdown"
    }
]

print('✅ Notebook initialized')
print(f'📦 Deployment name: {deployment_name}')
print(f'📍 Resource group: {resource_group_name}')
print(f'🌍 Location: {resource_group_location}')
print(f'🚀 MCP Servers to deploy: {len(mcp_servers)}')
for server in mcp_servers:
    print(f'   • {server["display_name"]} ({server["name"]})')


✅ Notebook initialized
📦 Deployment name: azure-architecture-toolkit
📍 Resource group: lab-azure-architecture-toolkit
🌍 Location: uksouth
🚀 MCP Servers to deploy: 3
   • AVM MCP Server (avm-modules)
   • Azure Pricing MCP Server (azure-pricing)
   • Markitdown MCP Server (markitdown)


### 1️⃣ Verify Azure CLI and connected subscription

Ensure you have the latest version of Azure CLI and are connected to your Azure subscription.

In [2]:
# Get current Azure account
output = run_command("az account show", "Retrieving Azure account information")
account_info = json.loads(output)

current_user = account_info['user']['name']
tenant_id = account_info['tenantId']
subscription_id = account_info['id']

print(f"✅ Connected to Azure")
print(f"👤 Current user: {current_user}")
print(f"🏢 Tenant ID: {tenant_id}")
print(f"📋 Subscription ID: {subscription_id}")


🔄 Retrieving Azure account information...
✅ Connected to Azure
👤 Current user: admin@MngEnvMCAP733116.onmicrosoft.com
🏢 Tenant ID: ac171e50-15dd-4b2f-a717-5d05ba421003
📋 Subscription ID: 789ff1bc-1521-4359-a12e-18eecb82f022


In [3]:
# Clone all MCP server repositories dynamically
print("🔄 Cloning MCP server repositories...")
for server in mcp_servers:
    try:
        run_command(
            f"git clone {server['repo_url']} ./{server['image_name']}", 
            f"Cloning {server['display_name']} repository"
        )
        print(f"✅ {server['display_name']} cloned to {server['image_name']}")
    except Exception as e:
        print(f"ℹ️ Repository {server['image_name']} may already exist or clone failed: {str(e)}")

print("✅ All repositories processed")


🔄 Cloning MCP server repositories...
🔄 Cloning AVM MCP Server repository...
✅ AVM MCP Server cloned to mcp-avm-modules
🔄 Cloning Azure Pricing MCP Server repository...
✅ Azure Pricing MCP Server cloned to mcp-azure-pricing
ℹ️ Repository mcp-markitdown may already exist or clone failed: 'repo_url'
✅ All repositories processed


### 2️⃣ Create resource group

Create the Azure resource group where all resources will be deployed.

In [4]:
# Check if resource group exists
check_rg = f"az group exists --name {resource_group_name}"
rg_exists = run_command(check_rg, "Checking if resource group exists")

if rg_exists.strip() == "true":
    print(f"ℹ️ Resource group '{resource_group_name}' already exists")
else:
    create_rg = f"az group create --name {resource_group_name} --location {resource_group_location}"
    run_command(create_rg, f"Creating resource group '{resource_group_name}'")
    print(f"✅ Resource group '{resource_group_name}' created successfully")

🔄 Checking if resource group exists...
🔄 Creating resource group 'lab-azure-architecture-toolkit'...
✅ Resource group 'lab-azure-architecture-toolkit' created successfully


### 3️⃣ Deploy infrastructure using Bicep

Deploy the Azure Container Registry, Container Apps Environment, and Container Apps using the main.bicep template.

In [5]:
# Create Bicep parameters file with dynamic server configuration
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "location": {"value": resource_group_location},
        "mcpServers": {
            "value": [
                {
                    "name": server["name"],
                    "displayName": server["display_name"],
                    "imageName": server["image_name"]
                }
                for server in mcp_servers
            ]
        }
    }
}

# Write parameters to file
with open('params.json', 'w') as f:
    json.dump(bicep_parameters, f, indent=2)

print("✅ Created parameters file with dynamic server configuration")
print(f"📋 Servers included: {len(mcp_servers)}")
for server in mcp_servers:
    print(f"   • {server['display_name']}")

# Deploy Bicep template
deploy_cmd = f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json"
run_command(deploy_cmd, f"Deploying infrastructure (this may take several minutes)")

print(f"✅ Deployment '{deployment_name}' completed successfully")


✅ Created parameters file with dynamic server configuration
📋 Servers included: 3
   • AVM MCP Server
   • Azure Pricing MCP Server
   • Markitdown MCP Server
🔄 Deploying infrastructure (this may take several minutes)...
✅ Deployment 'azure-architecture-toolkit' completed successfully


### 4️⃣ Get deployment outputs

Retrieve the outputs from the Bicep deployment to use in subsequent steps.

In [6]:
# Get deployment outputs
get_outputs_cmd = f"az deployment group show --name {deployment_name} --resource-group {resource_group_name} --query properties.outputs"
outputs_json = run_command(get_outputs_cmd, "Retrieving deployment outputs")
outputs = json.loads(outputs_json)

# Extract common values
container_registry_name = outputs['containerRegistryName']['value']
container_registry_login_server = outputs['containerRegistryLoginServer']['value']

# Extract dynamic MCP servers array
mcp_servers_output = outputs['mcpServers']['value']

# Create a lookup dictionary for easy access by server name
servers_lookup = {server['name']: server for server in mcp_servers_output}

# Display deployment outputs
print("✅ Retrieved deployment outputs:")
print(f"🐳 Container Registry: {container_registry_name}")
print(f"📦 MCP Servers deployed: {len(mcp_servers_output)}")

for server_output in mcp_servers_output:
    print(f"\n   {server_output['displayName']}:")
    print(f"   📦 Container App: {server_output['containerAppName']}")
    print(f"   🔗 Server URL: {server_output['url']}/mcp")


🔄 Retrieving deployment outputs...
✅ Retrieved deployment outputs:
🐳 Container Registry: acr7lsy3ktbwwv4k
📦 MCP Servers deployed: 3

   AVM MCP Server:
   📦 Container App: aca-avm-modules-7lsy3ktbwwv4k
   🔗 Server URL: https://aca-avm-modules-7lsy3ktbwwv4k.yellowgrass-fe297f24.uksouth.azurecontainerapps.io/mcp

   Azure Pricing MCP Server:
   📦 Container App: aca-azure-pricing-7lsy3ktbwwv4k
   🔗 Server URL: https://aca-azure-pricing-7lsy3ktbwwv4k.yellowgrass-fe297f24.uksouth.azurecontainerapps.io/mcp

   Markitdown MCP Server:
   📦 Container App: aca-markitdown-7lsy3ktbwwv4k
   🔗 Server URL: https://aca-markitdown-7lsy3ktbwwv4k.yellowgrass-fe297f24.uksouth.azurecontainerapps.io/mcp


In [7]:
# Install the containerapp extension
try:
    run_command("az extension add --name containerapp --upgrade", "Installing/upgrading containerapp extension")
    print("✅ Container Apps extension installed successfully")
except Exception as e:
    print(f"⚠️ Note: Extension may already be installed or installation failed: {str(e)}")
    print("Continuing with deployment...")

🔄 Installing/upgrading containerapp extension...
✅ Container Apps extension installed successfully


### 5️⃣ Build and deploy MCP server container images

Build the Docker images and push them to Azure Container Registry, then update the Container Apps.

In [9]:
build_number = build_number + 1

# Build and deploy all MCP servers dynamically
for server in mcp_servers:
    # Get the container app name from the servers lookup by matching server name
    server_output = servers_lookup[server['name']]
    containerapp_name = server_output['containerAppName']
    build_and_deploy_mcp_server(
        server, 
        containerapp_name, 
        resource_group_name, 
        container_registry_name, 
        container_registry_login_server, 
        build_number
    )

print("=" * 60)
print(f"🎉 All {len(mcp_servers)} MCP servers deployed successfully!")
print("=" * 60)


🏗️ Building AVM MCP Server
🔄 Building and pushing mcp-avm-modules:v2...
✅ AVM MCP Server image built: mcp-avm-modules:v2
🔄 Updating AVM MCP Server Container App...
✅ AVM MCP Server deployed successfully

🏗️ Building Azure Pricing MCP Server
🔄 Building and pushing mcp-azure-pricing:v2...
✅ Azure Pricing MCP Server image built: mcp-azure-pricing:v2
🔄 Updating Azure Pricing MCP Server Container App...
✅ Azure Pricing MCP Server deployed successfully

🏗️ Building Markitdown MCP Server
🔄 Building and pushing mcp-markitdown:v2...
✅ Markitdown MCP Server image built: mcp-markitdown:v2
🔄 Updating Markitdown MCP Server Container App...
✅ Markitdown MCP Server deployed successfully

🎉 All 3 MCP servers deployed successfully!


### 6️⃣ List available MCP tools

Connect to the MCP servers using the MCP protocol and list available tools.

In [2]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

print("=" * 60)
print("🔧 Discovering MCP Tools")
print("=" * 60)
print()

# List tools from all servers dynamically
all_tools = {}
for server_output in mcp_servers_output:
    tools = asyncio.run(list_mcp_tools(server_output['url'], server_output['displayName']))
    all_tools[server_output['name']] = tools

# Calculate total tools
total_tools = sum(len(tools) for tools in all_tools.values())

print("=" * 60)
print(f"✅ Total tools available: {total_tools}")
for server_output in mcp_servers_output:
    print(f"   • {server_output['displayName']}: {len(all_tools[server_output['name']])} tools")
print("=" * 60)


🔧 Discovering MCP Tools



NameError: name 'mcp_servers_output' is not defined

### Publish MCP server configuration for VSCode

In [1]:
# Generate Cline MCP configuration file
import os
from pathlib import Path

# Determine the home directory
home_dir = Path.home()
cline_config_dir = home_dir / '.config' / 'Code' / 'User' / 'globalStorage' / 'saoudrizwan.claude-dev' / 'settings'

# Create directory structure if it doesn't exist
cline_config_dir.mkdir(parents=True, exist_ok=True)

# Build Cline configuration with actual server URLs
cline_config = {
    "servers": {
        "avm-modules": {
            "type": "http",
            "url": f"{mcp_servers_output[0]['url']}/mcp"
        },
        "azure-pricing": {
            "type": "http",
            "url": f"{mcp_servers_output[1]['url']}/mcp"
        },
        "context7": {
            "type": "http",
            "url": "https://mcp.context7.com/mcp"
        },
        "markitdown": {
            "type": "http",
            "url": f"{mcp_servers_output[2]['url']}/mcp"
        }
    }
}

# Write the Cline configuration file
cline_config_path = cline_config_dir / 'cline_mcp_settings.json'
with open(cline_config_path, 'w') as f:
    json.dump(cline_config, f, indent=2)

print("✅ Cline MCP configuration file created")
print(f"📁 Location: {cline_config_path}")
print()
print("📋 Configuration:")
print(json.dumps(cline_config, indent=2))
print()
print("🎉 Your MCP servers are now configured for Cline in this Codespace!")

NameError: name 'mcp_servers_output' is not defined

In [11]:
# Generate MCP configuration file for VS Code dynamically
import os

# Read existing MCP configuration if it exists
mcp_config_path = '.vscode/mcp.json'
if os.path.exists(mcp_config_path):
    with open(mcp_config_path, 'r') as f:
        mcp_config = json.load(f)
    print("📖 Found existing MCP configuration file")
else:
    mcp_config = {"servers": {}}
    print("📝 Creating new MCP configuration file")

# Ensure servers key exists
if "servers" not in mcp_config:
    mcp_config["servers"] = {}

# Add all servers dynamically (updates existing or adds new)
for server_output in mcp_servers_output:
    mcp_config["servers"][server_output['name']] = {
        "type": "http",
        "url": f"{server_output['url']}/mcp"
    }

# Ensure .vscode directory exists
os.makedirs('.vscode', exist_ok=True)

# Write to mcp.json file
with open(mcp_config_path, 'w') as f:
    json.dump(mcp_config, f, indent=2)

print("✅ MCP configuration file created: .vscode/mcp.json")
print()
print("📋 MCP Server Configuration:")
print(json.dumps(mcp_config, indent=2))
print()
print("📝 To use with VS Code MCP Extension:")
print("   Copy the contents of mcp.json to:")
print("   ~/.config/Code/User/globalStorage/saoudrizwan.claude-dev/settings/cline_mcp_settings.json")
print()
print("🔗 Server URLs:")
for server_output in mcp_servers_output:
    print(f"   • {server_output['displayName']}: {server_output['url']}/mcp")


📖 Found existing MCP configuration file
✅ MCP configuration file created: .vscode/mcp.json

📋 MCP Server Configuration:
{
  "servers": {
    "microsoft-learn": {
      "type": "http",
      "url": "https://learn.microsoft.com/api/mcp"
    },
    "context7": {
      "type": "http",
      "url": "https://mcp.context7.com/mcp"
    },
    "avm-modules": {
      "type": "http",
      "url": "https://aca-avm-modules-7lsy3ktbwwv4k.yellowgrass-fe297f24.uksouth.azurecontainerapps.io/mcp"
    },
    "azure-pricing": {
      "type": "http",
      "url": "https://aca-azure-pricing-7lsy3ktbwwv4k.yellowgrass-fe297f24.uksouth.azurecontainerapps.io/mcp"
    },
    "markitdown": {
      "type": "http",
      "url": "https://aca-markitdown-7lsy3ktbwwv4k.yellowgrass-fe297f24.uksouth.azurecontainerapps.io/mcp"
    }
  }
}

📝 To use with VS Code MCP Extension:
   Copy the contents of mcp.json to:
   ~/.config/Code/User/globalStorage/saoudrizwan.claude-dev/settings/cline_mcp_settings.json

🔗 Server URLs:
  

### 🔧 Installing Azure MCP Server Extension in VS Code

You can also install the official **Azure MCP Server** extension directly in VS Code for enhanced Azure integration capabilities.

![Azure MCP Server Installation](https://github.com/microsoft/mcp/blob/main/servers/Azure.Mcp.Server/images/install_azure_mcp_server_extension.gif?raw=true)

**To install:**
1. Open VS Code Extensions panel (Ctrl+Shift+X / Cmd+Shift+X)
2. Search for "Azure MCP Server"
3. Click Install

For more details, visit: [Azure MCP Server Documentation](https://github.com/microsoft/mcp/blob/main/servers/Azure.Mcp.Server/README.md)

### 7️⃣ Usage Examples

#### Using with VS Code MCP Extension

Add these MCP servers to your VS Code configuration (`~/.config/Code/User/globalStorage/saoudrizwan.claude-dev/settings/cline_mcp_settings.json`):

```json
{
  "mcpServers": {
    "avm-modules": {
      "type": "http",
      "url": "YOUR_AVM_SERVER_URL/mcp"
    },
    "azure-pricing": {
      "type": "http",
      "url": "YOUR_PRICING_SERVER_URL/mcp"
    },
    "context7": {
      "type": "http",
      "url": "https://mcp.context7.com/mcp"
    },
    "markitdown": {
      "type": "http",
      "url": "YOUR_MARKITDOWN_SERVER_URL/mcp"
    }
  }
}
```

#### Using with Claude Desktop

Add to your Claude Desktop configuration:

```json
{
  "mcpServers": {
    "avm-modules": {
      "command": "http",
      "args": ["YOUR_AVM_SERVER_URL/mcp"]
    },
    "azure-pricing": {
      "command": "http",
      "args": ["YOUR_PRICING_SERVER_URL/mcp"]
    },
    "context7": {
      "command": "http",
      "args": ["https://mcp.context7.com/mcp"]
    },
    "markitdown": {
      "command": "http",
      "args": ["YOUR_MARKITDOWN_SERVER_URL/mcp"]
    }
  }
}
```

#### Example Queries

Once connected, you can ask:

**For AVM Modules:**
- "List all available Azure Verified Modules"
- "Show me details about the storage account AVM module"

**For Azure Pricing:**
- "What are the available Azure service families?"
- "How much does a Standard D2s v3 VM cost per month in East US?"
- "Compare pricing for Azure App Services in different regions"

**For Context7:**
- "Show me the latest Azure SDK for Python documentation"
- "Get code examples for using Azure Functions with Python"

**For MarkItDown:**
- "Convert this PDF document to Markdown"
- "Extract text from this Word document"

### 8️⃣ Clean up resources

When you're finished, remove all deployed resources to avoid unnecessary charges.

In [8]:
# Uncomment the following lines to delete the resource group and all resources
# WARNING: This will permanently delete all resources in the resource group!

# Ensure run_command is available (imported from utils in cell 1)
from utils import run_command

delete_rg_cmd = f"az group delete --name {resource_group_name} --yes --no-wait"
run_command(delete_rg_cmd, f"Deleting resource group '{resource_group_name}'")
print(f"🗑️ Resource group '{resource_group_name}' deletion initiated")
print("Note: Deletion may take several minutes to complete")

print("ℹ️ To delete all resources, uncomment the code above and run this cell")

🔄 Deleting resource group 'lab-azure-architecture-toolkit'...
🗑️ Resource group 'lab-azure-architecture-toolkit' deletion initiated
Note: Deletion may take several minutes to complete
ℹ️ To delete all resources, uncomment the code above and run this cell


---

## Summary

You've successfully deployed multiple MCP servers to Azure Container Apps:

1. **AVM MCP Server** - Provides tools to discover and query Azure Verified Modules
2. **Azure Pricing MCP Server** - Provides tools to query Azure pricing information
3. **Context7 MCP Server** - Access up-to-date library documentation (external service)
4. **MarkItDown MCP Server** - Convert documents to Markdown format

All deployed servers are now accessible via HTTPS and can be integrated with:
- VS Code Copilot with MCP extensions
- Claude Desktop
- Any MCP-compatible client
- Semantic Kernel applications

The deployment includes:
- ✅ Azure Container Registry for image storage
- ✅ Container Apps Environment with logging
- ✅ Three Container Apps running the MCP servers (AVM, Pricing, MarkItDown)
- ✅ Managed Identity for secure operations
- ✅ HTTPS ingress for external access
- ✅ External MCP service integration (Context7)

**Next Steps:**
- Configure your MCP client with the server URLs
- Consider installing the Azure MCP Server extension in VS Code
- Test the tools with your AI assistant
- Monitor logs in Log Analytics workspace
- Scale the Container Apps based on usage